# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación

### Minería de Datos: Curso académico 2020-2021

### Integrantes:

* Gonzalo Pinto Perez
* Yeremi Martin Huaman Torres


# 1. Preliminares

Cargamos las librerias que vamos a utilizar

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn import FunctionSampler
from imblearn.pipeline import make_pipeline
from scipy.stats import shapiro
from sklearn.compose import make_column_transformer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


# Local application
import miner_a_de_datos_an_lisis_exploratorio_utilidad as utils

Fijamos la semilla

In [ ]:
seed = 27912

Funcion que utilizaremos durante el transcurso de la práctica

In [ ]:
"Función para eliminar los datos anomalos"
def outlier_rejection(X, y):
    model = IsolationForest(max_samples=100,
                            contamination=0.4,
                            random_state=27912)
    model.fit(X)
    y_pred = model.predict(X)
    return X[y_pred == 1], y[y_pred == 1]

# 2. Acceso y almacenamiento de datos

Cargamos el conjunto de datos de la base de datos `pima-indians-diabetes-database` y `breast-cancer-wisconsin-data` :

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"
filepathWisconsin = "../input/breast-cancer-wisconsin-data/data.csv"

indexDiabetes = None
targetDiabetes = "Outcome"

indexWisconsin = "id"
targetWisconsin = "diagnosis"

dataDiabetes = utils.load_data(filepath, indexDiabetes, targetDiabetes)

dataWisconsin = utils.load_data(filepathWisconsin, indexWisconsin, targetWisconsin)

Comprobamos que hemos cargado bien los datos obteniendo una muestra no sesgada

In [ ]:
dataDiabetes.sample(5, random_state=seed)

In [ ]:
dataWisconsin.sample(5, random_state=seed)

Observamos que tenemos un dato incorrecto en dataWisconsin : "Unnamed:32" por lo tanto lo eliminamos.

In [ ]:
dataWisconsin = dataWisconsin.drop(dataWisconsin.columns[31], axis = 'columns')

Volvemos a comprobar 

In [ ]:
dataWisconsin.sample(5, random_state=seed)

Separamos el conjunto de datos en dos subconjuntos, uno con las variables predictoras (X) y otro con la variable objetivo(y)

In [ ]:
(XDiabetes, yDiabetes) = utils.divide_dataset(dataDiabetes, target="Outcome")

(XWisconsin, yWisconsin) = utils.divide_dataset(dataWisconsin, target="diagnosis")


Comprobamos que los datos se han separado correctamente, primero las variables predictoras

In [ ]:
XDiabetes.sample(5, random_state=seed)

In [ ]:
XWisconsin.sample(5, random_state=seed)

Y ahora la variable clase

In [ ]:
yDiabetes.sample(5, random_state=seed)

In [ ]:
yWisconsin.sample(5, random_state=seed)

Para realizar el proceso de holdout tenemos que dividir el conjunto de datos entre dos subconjuntos, uno que sirva de muestra de entrenamiento (el 70%) y otro que sirva de muestra de prueba (el 30%).

In [ ]:
train_size = 0.7

(XDiabetes_train, XDiabetes_test, yDiabetes_train, yDiabetes_test) = train_test_split(XDiabetes, yDiabetes,
                                                      stratify=yDiabetes,
                                                      random_state=seed,
                                                      train_size=train_size)
(XWisconsin_train, XWisconsin_test, yWisconsin_train, yWisconsin_test) = train_test_split(XWisconsin, yWisconsin,
                                                      stratify=yWisconsin,
                                                      random_state=seed,
                                                      train_size=train_size)


Comprobamos que se han separado correctamente las variables predictoras:

In [ ]:
XDiabetes_train.sample(5, random_state=seed)

In [ ]:
XDiabetes_test.sample(5, random_state=seed)

In [ ]:
XWisconsin_train.sample(5, random_state=seed)

In [ ]:
XWisconsin_test.sample(5, random_state=seed)

Comprobamos que se han separado correctamente las variables clase:

In [ ]:
yDiabetes_train.sample(5, random_state=seed)

In [ ]:
yDiabetes_test.sample(5, random_state=seed)

In [ ]:
yWisconsin_train.sample(5, random_state=seed)

In [ ]:
yWisconsin_test.sample(5, random_state=seed)

# 3. Analisis exploratorio, preprocesamiento de datos, algoritmos de clasificación y evaluacion de modelos de la base datos pima indians diabetes

# 3.1 Análisis exploratorio de datos de la base de datos pima indians diabetes

Para facilitar el análisis exploratorio volvemos a juntar las variables predictoras con las variables clases tanto con el conjunto de datos de prueba como el conjunto de datos de entrenamiento:

In [ ]:
dataDiabetes_train = utils.join_dataset(XDiabetes_train, yDiabetes_train)
dataDiabetes_test = utils.join_dataset(XDiabetes_test, yDiabetes_test)

Lo primero que hacemos es ver el numero de casos y variables del problema:

In [ ]:
dataDiabetes_train.shape

Tras esto podemos observar que este problema tiene excesivo número de casos, lo cual puede suponer un inconveniente para la creación de modelos y su posterior evaluación.

Obtenemos más información sobre el problema que nos podría ser de utilidad posteriormente como el tipo de las variables:

In [ ]:
dataDiabetes_train.info(memory_usage=False)

Los distintos estados que puede tener de la variable clase:

In [ ]:
yDiabetes.cat.categories

Empezamos realizando un histograma de los datos:

In [ ]:
utils.plot_histogram(dataDiabetes_train)

Este primer gráfico nos muestra datos muy relevantes. Lo primero en lo que nos fijamos es en la distribución que tienen las variables en este gráfico, destacando a simple vista las que parecen que tienen una distribución normal con tendencia central en forma de campana de gaus, las cuales son: Glucose, BloodPresure y BMI. Esto puede suponer en un principio que estas variables serán más fáciles de tratar en el preprocesamiento de datos pero si nos fijamos más de cerca podemos darnos cuenta de que los valores mínimos de estas variables son o 0 o en algunas situaciones valores negativos inferiroes a 0, valores que no tienen sentido para estas variables, por lo tanto en el procesamiento de datos lo primero que tendremos que hacer será reemplazar los ceros de estas variables por valores viables.
A continuación si nos fiajamos en el resto de la variables(Pregnancies, SkinThickness, Insulin,DiabetesPedigreeFunction y Age), y nos damos cuenta de que todas ellas contienen una distribución con tendencia exponencial decreciente o de otra manera, que sigue una distribución sesgada positivamente, esta distribución es bastante problemática ya que al extenderse con un amplio siesgo y al haber un gran casos (como hemos comentado anteriormente), el conjunto de datos tendrá una cantidad considerable de outliers o datos anómales de los cuales tendremos que eliminar algunos en el preprocesamiento para que los modelos resultantes tenga una precisión mejor. Además si analizamos estas últimas variables podremos descubrir que las variables SkinThickness y Insulin vuelven a tener la misma problemática que hemos comentado con anterioridad, estas variables pueden tener el valor 0 aunque por lógica no lo deberían de aceptar, por lo tanto tendremos que reemplazar los ceros en estas variables al igual que en las mencionadas anteriormente.

Posteriormente realizamos un diagrama de barras de los datos para comprobar la distribución de la variable objetivo:

In [ ]:
utils.plot_barplot(dataDiabetes_train)

A continuación realizamos una matriz de gráficos del tipo nube puntos para comprobar la distribución de los datos:

In [ ]:
utils.plot_pairplot(dataDiabetes_train, target="Outcome")

Tras observar la matriz de gráficos del tipo nube puntos podemos comprobar lo que ibamos observando en los anteriores gráficos, hay un exceso de casos que nos impiden observar correctamente la distribución de la variable objetivo para realizar correctamente una discretización de forma correcta. 

Para acabar con los gráficos, realizaremos un gráfico de matriz de correlación para comprobar si tenemos que eliminar alguna de las variables predictoras del conjunto de datos:

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(dataDiabetes_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Observando el gráfico podemos llegar a la conclusión de que ninguna el máximo nivel de correlación que tenemos entre dos variables es el de BMI y SckinThickness, siendo este 0.6, no obstante no lo consideramos lo suficientemente alto como para eliminar una de las dos variables antes durante el preprocesamiento de datos antes de realizar la generación de los modelos.

A continuación seguimos realizando el análisis exploratorio analizando las variables numericas:

In [ ]:
dataDiabetes.describe(include="number")

Podemos observar que hay 768 datos, y lo más posible esque gran parte de esos casos los podramos menospreciar para desarrolar la calificación del modelo resultante

Y seguimos analizando las variables categóricas o en este caso la variable objetivo:

In [ ]:
dataDiabetes.describe(include="category")

Esta salida corrobora las conclusiones que hemos sacado de lo que llevamos de análisis, pues nos indica que hay demasiados casos y que la variable 0 se repite demasiado, ya que su frecuencia es de 500 teniendo en cuenta que hay 750 casos, y de esta manera no estaríamos ante un problema balanceado.

# 3.2 Preprocesamiento de datos de la base de datos pima indians diabetes

Limpieza de datos:

Primero eliminamos los ceros de aquellas variables (recordar que dichas variables eran Bloodpresure, BMI, Glucose, Insulin y SkinThickness) donde hemos detectado ceros y que no deberian de tenerlo. Para ello donde haya un cero, sustituiremos el cero por la media que tenga esa variable predictora con respecto a la variable objetivo:

In [ ]:
df1 = dataDiabetes.loc[dataDiabetes['Outcome'] == 1]
df2 = dataDiabetes.loc[dataDiabetes['Outcome'] == 0]
df1 = df1.replace({'BloodPressure':0}, np.median(df1['BloodPressure']))
df2 = df2.replace({'BloodPressure':0}, np.median(df2['BloodPressure']))
dataframe = [df1, df2]
dataDiabetes = pd.concat(dataframe)

df1 = dataDiabetes.loc[dataDiabetes['Outcome'] == 1]
df2 = dataDiabetes.loc[dataDiabetes['Outcome'] == 0]
df1 = df1.replace({'BMI':0}, np.median(df1['BMI']))
df2 = df2.replace({'BMI':0}, np.median(df2['BMI']))
dataframe = [df1, df2]
dataDiabetes = pd.concat(dataframe)

df1 = dataDiabetes.loc[dataDiabetes['Outcome'] == 1]
df2 = dataDiabetes.loc[dataDiabetes['Outcome'] == 0]
df1 = df1.replace({'Glucose':0}, np.median(df1['Glucose']))
df2 = df2.replace({'Glucose':0}, np.median(df2['Glucose']))
dataframe = [df1, df2]
dataDiabetes = pd.concat(dataframe)

df1 = dataDiabetes.loc[dataDiabetes['Outcome'] == 1]
df2 = dataDiabetes.loc[dataDiabetes['Outcome'] == 0]
df1 = df1.replace({'Insulin':0}, np.median(df1['Insulin']))
df2 = df2.replace({'Insulin':0}, np.median(df2['Insulin']))
dataframe = [df1, df2]
dataDiabetes = pd.concat(dataframe)

df1 = dataDiabetes.loc[dataDiabetes['Outcome'] == 1]
df2 = dataDiabetes.loc[dataDiabetes['Outcome'] == 0]
df1 = df1.replace({'SkinThickness':0}, np.median(df1['SkinThickness']))
df2 = df2.replace({'SkinThickness':0}, np.median(df2['SkinThickness']))
dataframe = [df1, df2]
dataDiabetes = pd.concat(dataframe)


Despues utilizamos Synthteic Minority Oversampling Technigque (SMOTE) para eliminar el desbalanceo que existe en los datos de entrenamiento del problema creando muestran usando los datos de entrenamiento actuales.

In [ ]:
smt = SMOTE()
XDiabetes_train, yDiabetes_train = smt.fit_sample(XDiabetes_train, yDiabetes_train)
dataDiabetes_train = utils.join_dataset(XDiabetes_train, yDiabetes_train)

Comprobamos que hemos corregido el desbalanceo original, realizando de nuevo un de diagramas de barras

In [ ]:
utils.plot_barplot(dataDiabetes_train)

Y al analizarlo podemos concluir que finalmente los datos de entrenamiento del problema se han balanceado.

A continuación vamos a escalar nuestros datos para tratar con datos anómalos u outliers para hacer que los datos anómalos estén menos sesgados con respecto a los otros. Para ello utilizaremos StandarScaler que "escala" la propiedad restando por la media y diviendo por la desviación estándar

In [ ]:

yDiabetes = dataDiabetes.Outcome
XDiabetes = dataDiabetes.drop('Outcome', axis = 1)
columns = XDiabetes.columns
scaler = StandardScaler()
XDiabetes = scaler.fit_transform(XDiabetes)
dataDiabetes_x = pd.DataFrame(XDiabetes, columns = columns)

Una vez escalada, volvemos a dividir el conjunto de datos inicial entre datos de entrenamiento y de test, para generar estos dos subconjuntos pero sin datos anómalos.

In [ ]:
XDiabetes_train, XDiabetes_test, yDiabetes_train, yDiabetes_test = train_test_split(dataDiabetes_x, yDiabetes,
                                                                    stratify=yDiabetes, random_state = seed, train_size = train_size)

Y a continuación vamos a realizar la discretización. Como en el diagrama de tipos puntos de nube no podemos tener una referencia clara del punto o la forma de realizar la discretización vamosa  realizar la discretización de tres formas diferentes; de igual anchura, de igual profundidad y en k-medias. Obviamente, estas tres discretizaciones del problema y al tener este una variable objetivo de tipo categorico pudiendo tener dos valores (0 o 1), todas las discretizaciones tendrán dos intervalos pues intentamos que en cada interavalo estén agrupados el mayor número de variables de un único tipo posible. Estas discretizaciones son las siguientes:

In [ ]:

discretizer2u = KBinsDiscretizer(n_bins=2, strategy="uniform")
discretizer2q = KBinsDiscretizer(n_bins=2, strategy="quantile")
discretizer2k = KBinsDiscretizer(n_bins=2, strategy="kmeans")


#  3.3 Algoritmos de clasificación de la base de datos pima indians diabetes

A continuación vamos a generar los distintos modelos basados en los 2 algoritmos que nos pide inicialmente el enunciado de la práctica.

*Algoritmo Zero-R:*

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

*Algoritmo Cart o algoritmo de clasificación y regresión de árboles:*

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

Y al árbol de regresión le aplicamos las tres discretizaciones que hemos preparado:

In [ ]:
discretizeUniform_tree_model = make_pipeline(discretizer2u, tree_model)

In [ ]:
discretizeQuantile_tree_model = make_pipeline(discretizer2q, tree_model)

In [ ]:
discretizeKmeans_tree_model = make_pipeline(discretizer2k, tree_model)

A continuación, si el rescalado de los datos anómalos no ha sido suficiente, vamos a aplicar una pipeline a los algoritmos ya desarrollados que utiliza una función basada en el algoritmo IsolationForest que se encargará de eliminar los posibles datos outliers o anómalos que queden en los datos y en la parte de evaluación nos encargaremos de analizar los resultados obtenidos y comapararnos con la versión de los algoritmos sin dicha pipeline.

*Algoritmo zero_r_model con eliminación de outlier o datos anómalos con anterioridad*

In [ ]:
zero_r_modelPipe = make_pipeline(FunctionSampler(func=outlier_rejection),zero_r_model)

*Algoritmo tree_mode con eliminación de outlier o datos anómalos con anterioridad*

In [ ]:
tree_modelPipe = make_pipeline(FunctionSampler(func=outlier_rejection),tree_model)

*Algoritmo tree_mode con discretización y eliminación de outlier o datos anómalos con anterioridad*

In [ ]:
discretizeUniform_tree_modelPipe = make_pipeline(FunctionSampler(func=outlier_rejection),discretizeUniform_tree_model)

In [ ]:
discretizeQuantile_tree_modelPipe= make_pipeline(FunctionSampler(func=outlier_rejection),discretizeQuantile_tree_model)

In [ ]:
discretizeKmeans_tree_modelPipe = make_pipeline(FunctionSampler(func=outlier_rejection),discretizeKmeans_tree_model)

Y finalmente, hemos aprovechado el tipo de problema que supone esta base de datos con datos anómalos y la función que tenemos para implementar pipelines que eliminen con anterioridad los datos anómalos para implementar tres tipos de algoritmos de ajuste de modelos: Regresión logística, Máquina de vector de soporte o SVC y el modelo random forest.

*Pipeline creado para aplicar el modelo de regresión logística*

In [ ]:
LogisticRegresionPipe = make_pipeline(FunctionSampler(func=outlier_rejection),
                     LogisticRegression(solver='lbfgs', multi_class='auto',
                                        random_state=27912))

*Pipeline creado para aplicar el modelo de la máquina de vector soporte:*

In [ ]:
SupportVectorMachinePipe= make_pipeline(FunctionSampler(func=outlier_rejection),
                     SVC(kernel = 'rbf',random_state=27912))

*Pipeline creado para aplicar el modelo de random forest:*

In [ ]:

RandomForestPipe= make_pipeline(FunctionSampler(func=outlier_rejection),
                     RandomForestClassifier(n_estimators=300, bootstrap = True, max_features = 'sqrt',random_state=27912))

#  3.4 Evaluación de modelos de la base de datos pima indians diabetes

Evaluación modelo Zero-R

In [ ]:
utils.evaluate(zero_r_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

Como era de esperar el modelo Zero-R al ser el modelo más tribial da una precisión que de aún sin ser mala del todo, es bastante mejorable aún tras haber realizado todo el preprocesamiento de datos

Evaluación del modelo CART o algoritmo de regresión de árboles

In [ ]:
utils.evaluate(tree_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

En este algoritmo notamos finalmente la primera mejora del preprocesamiento de datos teniendo una gran mejora en la precisión del modelo

Evalución del modelo CART discretizado de tres maneras distintas

In [ ]:
utils.evaluate(discretizeUniform_tree_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)


In [ ]:
utils.evaluate(discretizeQuantile_tree_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)


In [ ]:
utils.evaluate(discretizeKmeans_tree_model,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)


Tras observar la precisión de los modelos y tras realizarle las tres discretizaciones posibles podemos observar que de todas las discretizaciones la que ofrece la mejor precisión es la discretización en frecuencia y esto es debido a que los procesos realizados durante el preprocesamiento de datos han reducido el siesgo de los datos y han convertido este problema en un problema balanceado, intentando "juntar" las variables predictoras y objetivos de los casos en las que las categorias de las variable objetivo son iguales.

*Evaluación de con el pipeline de reducción de datos anómalos aplicado*

Evaluación del modelo Zero-R

In [ ]:
utils.evaluate(zero_r_modelPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

Como era de esperar el modelo Zero-R aún aplicando un pipeline que deberia de mejorar su redimiento, es un algoritmo tan trivial que ni consigue mejorar su precisión ni empeorarla, simplemente se queda igual

Evaluación del modelo CART o algoritmo de regresión de árboles

In [ ]:
utils.evaluate(tree_modelPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

Tras aplciarle el pipeline, el algoritmo CART ha obtenido incluso una mejor precisión que la que obtuvo sin aplicarsela debido a que está a eliminado incluso más outliers o datos anómalos de los que ya había en los datos antes de aplicarle el preprocesamiento de datos.

Evalución del modelo CART discretizado de tres maneras distintas

In [ ]:
utils.evaluate(discretizeUniform_tree_modelPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.evaluate(discretizeQuantile_tree_modelPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

In [ ]:
utils.evaluate(discretizeKmeans_tree_modelPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

Tras observar los resultados de estas tres discretizaciones junto con la aplicación de la pipeline podemos sacar ciertas conclusiones importantes.La discretización basada en anchura al ser una discretización que se basa en los máximos y los mínimos al emplear la pipeline y eliminar aún más outliers o datos anómalos es la que más ha conseguido mejorar la precisión del modelo obtenido. Mientras tanto, las otras dos variables no se ven afectadas demasiado por la eliminación extra de los outleirs o datos anómalos utilizando la piple, aunque la discretización con mejores resultados sigue siendo la basada en frecuencia,debido a lo argumentado en apartados anteriores, pues en el preprocesamiento de datos se ha intentado eliminar el sesgo entre datos.

*Algorimtos de ajustes de modelos*

Regresión lineal

In [ ]:
utils.evaluate(LogisticRegresionPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

Máquina del vector soporte

In [ ]:
utils.evaluate(SupportVectorMachinePipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

RandomForest

In [ ]:
utils.evaluate(RandomForestPipe,
               XDiabetes_train, XDiabetes_test,
               yDiabetes_train, yDiabetes_test)

Finalmente tras ver las precisiones obtenidas por estos tres algoritmos podemos concluir que aunque en el enunciado inicial se nos pidiese únicamente la aplicación de los algoritmos Zero-R y CART, el uso de algoritmos para generar modelos basados en el ajuste de modelos es el más recomendable para estos problemas con outliers o datos anómalos; ya que estos tres algoritmos han obtenido muy buenas preciones en sus modelos superando a la precisión que da el modelo del algoritmo Zero-R, y el último algoritmo, RandomForest, incluso ha conseguido obtener una precisión similar a la que da el algoritmo CART sin aplicarle la pipeline.

# 4. Análisis exploratorio, preprocesamiento de datos, algoritmos de clasificación y evaluacion de modelos de la base datos breast cancer wisconsin data



# 4.1 Análisis exploratorio de la base datos breast cancer wisconsin data

Antes que nada obtendremos el conjunto de datos de entrenamiento y el de prueba

In [ ]:
dataWisconsin_test= utils.join_dataset(XWisconsin_test,yWisconsin_test)
dataWisconsin_train= utils.join_dataset(XWisconsin_train,yWisconsin_train)

**Descripcion del conjunto**

Tendremos que tener conocimento de:
* Numeros de casos
* Tipos de variables

In [ ]:
dataWisconsin_train.shape

Obsevamos que el conjunto de datos de entrenamiento tenemos 398 casos y 31 variables (31 variables predictoras y 1 variable clase). 

Para conocer el tipo de variable 

In [ ]:
dataWisconsin_train.info(memory_usage=False)

Observamos que tenemos que las 31 seran de tipo numerico (float64) y la variable diagnosis es categorica esta contendra los siguientes casos:

In [ ]:
yWisconsin_train.cat.categories

**Vizualizacion de la variables**

Debemos representar y analizar las distribuciones de las variables.

In [ ]:
utils.plot_histogram(dataWisconsin_train)

En este histograma muestra la densidad de las instancias para las diferencia variables.

Muestran una distribucion normal
una distribución normal con tendencia central en forma de campana, la mayoria de caracteriticas.

In [ ]:
utils.plot_barplot(dataWisconsin_train)
B,M= yWisconsin_train.value_counts()
print('Numero de Benig: ',B)
print('Numero de Malignant : ',M)

Observando el conjunto de datos de entrenamiento tenemos 250 Benig y 148 Malignant, tenemos dos variables objetivos y que no tenemos el misma numero de casos, esto quiere decir que la muestra esta desbalanceada.

Al tener muchas caracteristicas resulta dificil poder visualizar algunas graficas por ejemplo el diagrama de nubes de puntos, por ello divimos estas caracteristicas en 3, respectivamente(mean, re y worst)

In [ ]:
utils.plot_pairplot(dataWisconsin_train, target="diagnosis")

In [ ]:
mean_train = dataWisconsin_train.iloc[:,[0,1,2,3,4,5,6,7,8,9,30]]
se_train = dataWisconsin_train.iloc[:,[10,11,12,13,14,15,16,17,18,19,30]]
worst_train = dataWisconsin_train.iloc[:,[20,21,22,23,24,25,26,27,28,29,30]]
utils.plot_pairplot(mean_train, target="diagnosis")

In [ ]:
utils.plot_pairplot(se_train, target="diagnosis")

In [ ]:
utils.plot_pairplot(worst_train, target="diagnosis")

Respecto a estas graficas podemas darnos cuenta que las variables radius_mean, perimeter_mean, area_mean estan muy relacionados, obviamente es puede ser por que para obtener el perimetro y el area es necesario saber el radio. Puede que sea factible descartar estas variables y quedarnos con solo radio.

Podemos ver en la matriz de correlacion observamos que radio, perimetro, area si estan muy correlacionados, ademas de concavidad, compactness y concave_point, por lo que podemos utilizar uno de elllos y descartar los otros.

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(mean_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(se_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(worst_train.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

# 4.2 Preprocesamiento de datos de la base datos breast cancer wisconsin data

**Transformacion de los datos:**

Como hemos observado en el analisis exploratorio, una accion posible es la eliminacion de variables, por no resultar muy util o redundante.

Entonces que datos descartar: Nos quedaremos con radius_mean por ejemplo entre area_mean,perimeter_mean,radius_mean y tambien con concavidad entre (concavidad,compactness,concave_points) en cada caso lo mismo (mean,worst,se) Por lo tanto tendremos de las 30 variables, entonces tendremos 18 variables a utilizar.

Creamos una lista de nombres de las caracteristicas con intencion de quitarlas:


In [ ]:
droplist= ['perimeter_mean','area_mean','perimeter_se','area_se','perimeter_worst','area_worst','concave points_mean','compactness_mean',
          'concave points_se','compactness_se','concave points_worst','compactness_worst']
newdataWisconsin= dataWisconsin.drop(droplist,axis=1)

newdataWisconsin.sample(5,random_state=seed)

Comprobamos que se hallan borrado 

In [ ]:
newdataWisconsin.sample(5,random_state=seed)

A partir de estos datos realizamos el holdout:
    

In [ ]:
(nXW, nyW) = utils.divide_dataset(newdataWisconsin, target="diagnosis")
train_size = 0.7

(nXW_train, nXW_test, nyW_train, nyW_test) = train_test_split(nXW, nyW,
                                                      stratify=nyW,
                                                      random_state=seed,
                                                      train_size=train_size)
newdataW_test= utils.join_dataset(nXW_test,nyW_test)
newdataW_train= utils.join_dataset(nXW_train,nyW_train)


Probaremos si discretizar es un opcion viable para este conjunto de datos, por lo que creamos los discretizadores (anchura,frecuencia,k-medias):

In [ ]:
discretizer2uW = KBinsDiscretizer(n_bins=2, strategy="uniform")
discretizer2qW = KBinsDiscretizer(n_bins=2, strategy="quantile")
discretizer2kW = KBinsDiscretizer(n_bins=2, strategy="kmeans")


#  4.3 Algoritmos de clasificación de la base datos breast cancer wisconsin data

Generamos los distintos clasificadores:

- Algoritmo Zero_R
- Algoritmo CART

Tambien crearemos un estimador que permite integrar la eliminacion de atributos dentro de un pipeline (delete_transformer). Este realizara la misma funcion que se hizo en el prepocesamiento (Transformacion de los datos).

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed)

In [ ]:
delete_transformer = make_column_transformer(('drop',['perimeter_mean','area_mean','perimeter_se','area_se','perimeter_worst','area_worst','concave points_mean','compactness_mean',
          'concave points_se','compactness_se','concave points_worst','compactness_worst']),remainder='passthrough')

Tambien crearemos distintos pipelines para compraborar si las mejoras que realizamos resultan correctas o no.

In [ ]:
delete_zeroR=make_pipeline(delete_transformer, zero_r_model)

In [ ]:
delete_treemodel=make_pipeline(delete_transformer, tree_model)

In [ ]:
discretize_tree_modeluW = make_pipeline(discretizer2uW,tree_model)

In [ ]:
discretize_tree_modelqW = make_pipeline(discretizer2qW, tree_model)

In [ ]:
discretize_tree_modelkW = make_pipeline(discretizer2kW, tree_model)

In [ ]:
delete_treemodel_discretizeu=make_pipeline(delete_transformer,discretizer2uW,tree_model)

In [ ]:
delete_treemodel_discretizeq=make_pipeline(delete_transformer,discretizer2qW,tree_model)

In [ ]:
delete_treemodel_discretizek=make_pipeline(delete_transformer,discretizer2kW,tree_model)

#  4.4 Evaluación de modelos de la base datos breast cancer wisconsin data

Primero evaluaremos el Algoritmo-Zero-R 

In [ ]:
utils.evaluate(zero_r_model,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

Probamos si eliminando los valores redundantes resulta en alguna mejora

In [ ]:
utils.evaluate(delete_zeroR,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

Como era de esperar el modelo Zero-R al ser el modelo más malo, ya que considera a todos los datos como Benignos e introduciendo la eliminacion de datos no resulta de ayuda.

Ahora probemos con el arbol de decision:

In [ ]:
utils.evaluate(tree_model,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

Resulta en una gran mejora con respecto al Zero-R, probemos ahora si eliminando caracteristicas es correcto.

In [ ]:
utils.evaluate(delete_treemodel,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

Resulta en una mejor evaluacion.
Probemos con el conjunto de datos discretizado en el arbol de decision  en cada caso (anchura,frecuencia,k-medias) respecticamente.

In [ ]:
utils.evaluate(discretize_tree_modeluW,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)



In [ ]:
utils.evaluate(discretize_tree_modelqW,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.evaluate(discretize_tree_modelkW,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

Observamos que la evaluacion para anchura y frencuencia no mejora con respecto al conjunto sin discretizar, pero por k-medias si.

Entonces que pasaria si utilizamos sobre estos pipeline la eliminacion de caracteristicas redundantes.

In [ ]:
utils.evaluate(delete_treemodel_discretizeu,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.evaluate(delete_treemodel_discretizeq,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

In [ ]:
utils.evaluate(delete_treemodel_discretizek,
               XWisconsin_train, XWisconsin_test,
               yWisconsin_train, yWisconsin_test)

Finalmente tras ver las precisiones obtenidas son las siguientes:

- Zero_R: 0.62573 sin eliminacion y con eliminacion son la misma
- Arbol de decision: 0.90643 sin eliminacion y 0.94737 con eliminacion
- Discretizacion + Arbol de decision:

    1. Anchura : 0,90058 sin eliminacion y 0.80702 con eliminacion
    2. Frecuencia: 0.89474 sin eliminacion y 0.88304 con eliminacion
    3. k-medias: 0.92398 sin eliminacion y con eliminacion misma
    
En estas tres opciones podemos decir que el que nos da mayor resultado es arbol de decision + eliminacion con un 0.94737 y el peor Zero_R, pero obviando este el siguiente seria discretizacion(anchura) + eliminacion + arbol de decision con 0.80702.

Observando detenidamente los resultados la discretizacion sobre las caracteristicas eliminadas resulta en un considerable descendo en la precision por lo tanto no seria correcto discretizar cuando se eliminan caracteristicas.

Por otro lado si solo discretizamos(k-medias) + arbol de decision resultamos en un 0.92398 que resulta en buena precision en comparacion con las otras discretizaciones y tambien en el caso de realizar el arbol de decision sin discretizar.